In [1]:
import pandas as pd

# read in

In [2]:
df = pd.read_csv('../../2_Feature_Engineering/export/combined_polly_bretschneider_iwg.csv')
df.head()

,raw_text,severity,binary_label
0,ist ekelhaft bei ihren Klos kannst du nur mit...,0.0,False
1,Bevor Sie mit Jud...,1.0,True
2,Und de Amis freut es wenn die Heuschrecken Eur...,0.0,False
3,": Die Krim ist ein heißer Tip, auch zum In...",1.0,True
4,: Es muss in #Deutschland eine politische Kra...,1.0,True


# clean

In [3]:
import sys
sys.path.insert(0, r'..\..\0_common')
from model_helpers import clean_all as ci

In [4]:
df['cleaned_text'] = df['raw_text'].apply(ci)
df.head()

,raw_text,severity,binary_label,cleaned_text
0,ist ekelhaft bei ihren Klos kannst du nur mit...,0.0,False,ekelhaft klos kannst gummistiefel gehen stinkt...
1,Bevor Sie mit Jud...,1.0,True,bevor judenstern markiert worden juden ganze ...
2,Und de Amis freut es wenn die Heuschrecken Eur...,0.0,False,de amis freut heuschrecken europa klein wirtsc...
3,": Die Krim ist ein heißer Tip, auch zum In...",1.0,True,krim heißer tip investieren muß bürger landes...
4,: Es muss in #Deutschland eine politische Kra...,1.0,True,deutschland politische kraft geben sozial sch...


In [5]:
df.shape

(9169, 4)

# vectorize

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

In [7]:
vec = CountVectorizer(ngram_range=(3,5), analyzer='char_wb', max_features=100000)

In [8]:
X_features = vec.fit_transform(df['cleaned_text'])

In [9]:
X_features.shape

(9169, 74355)

# initialize model

In [26]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(solver='liblinear', penalty='l2', C=0.23357214690901212) # params from earlier grid search

# Cross validation

In [31]:
from sklearn.model_selection import cross_validate
scores = cross_validate(clf, X_features, df['binary_label'], cv=5,
                        scoring=('f1', 'roc_auc', 'precision', 'recall'),
                        return_train_score=True)

In [32]:
sorted(scores.keys())

['fit_time',
 'score_time',
 'test_f1',
 'test_precision',
 'test_recall',
 'test_roc_auc',
 'train_f1',
 'train_precision',
 'train_recall',
 'train_roc_auc']

In [33]:
scores_df = pd.DataFrame(scores)
scores_df

,fit_time,score_time,test_f1,train_f1,test_roc_auc,train_roc_auc,test_precision,train_precision,test_recall,train_recall
0,0.673151,0.010982,0.902810,0.992885,0.953503,0.999755,0.913507,0.996212,0.892361,0.989580
1,0.756977,0.010970,0.909305,0.992739,0.958194,0.999817,0.925659,0.996211,0.893519,0.989291
2,0.717080,0.009973,0.895981,0.991557,0.958750,0.999748,0.915459,0.997365,0.877315,0.985818
3,0.684167,0.012966,0.911124,0.991995,0.967356,0.999842,0.926946,0.997658,0.895833,0.986397
4,0.850725,0.010969,0.912565,0.993318,0.964706,0.999814,0.912037,0.997375,0.913094,0.989294


In [36]:
# metrics
scores_df[['test_f1', 'test_roc_auc', 'test_precision', 'test_recall']].mean()

test_f1           0.906357
test_roc_auc      0.960502
test_precision    0.918722
test_recall       0.894424
dtype: float64